In [154]:
from lxml import html
from re import search
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs4

In [167]:
# BASE URL FOR CATEGORY
BASE_URL_CATEGORY = "https://commons.wikimedia.org/wiki/Category:Landscapes"

# BASE URL FOR WIKIMEDIA
BASE_URL_WIKIMEDIA = "https://commons.wikimedia.org"

In [168]:
def get_download_path(base_url):
  """
    The goal of this function is to get the downloadable paths
    to use in the scrape_images function.
    You are going to get all the path for each single image that you'll
    use after with the scrape_images function.
    
    From https://www.simplilearn.com/tutorials/python-tutorial/python-internet-access-using-urllib-request-and-urlopen#:~:text=The%20urlopen()%20function%20provides,objects%20that%20perform%20these%20services.:
    
    The urlopen() function provides a fairly simple interface. 
    It is capable of retrieving URLs with a variety of protocols. 
    It also has a little more complicated interface for dealing with typical 
    scenarios, such as basic authentication, cookies, and proxies. 
    Handlers and openers are objects that perform these services.
  """

  # FETCH THE BASE URL
  html = urlopen(base_url)
  bs4_apply = bs4(html, 'html.parser')
  # FIND ALL ELEMENTS WITH THE CLASS galleryfilename galleryfilename-truncate
  link_images = bs4_apply.find_all('a', {"class": "galleryfilename galleryfilename-truncate"})

  save_href_list = []

  for el in link_images:
    get_ref = el.get('href')
    save_href_list.append(BASE_URL_WIKIMEDIA + get_ref)
    # print(el.get('href'))

  print(save_href_list)

In [169]:
get_download_path(BASE_URL_CATEGORY)

['https://commons.wikimedia.org/wiki/File:Crest-_Yosemite%3F_(17a34fd143a7446ea27c2138f52c2e16).jpg', 'https://commons.wikimedia.org/wiki/File:Fallcreek_Oct_2008.jpg', 'https://commons.wikimedia.org/wiki/File:IMG-20210214-WA0300.jpg', 'https://commons.wikimedia.org/wiki/File:In_Bangor.JPG', 'https://commons.wikimedia.org/wiki/File:Joe_Mania_2016-08-01_(Unsplash_PFLchrsv9jY).jpg', 'https://commons.wikimedia.org/wiki/File:Layakharo_Evining_sunset_scenery.jpg', 'https://commons.wikimedia.org/wiki/File:Mountain_Glacier_Landscape_(from_AvoPix).jpg', 'https://commons.wikimedia.org/wiki/File:%D0%AF%D1%8F_412a.jpg']


In [174]:
def scrape_images(download_path):
  """
  Using the download_path url this function fetch
  the correct url and download the image in the 
  current working directory.
  """
  # GET THE FILE NAME IN ORDER TO USE IT LATER TO SAVE THE FILE
  file_name = download_path.split("File:")[1] # TO CHECK: IS THIS A STRING?
  html = urlopen(download_path)
  bs4_apply = bs4(html, 'html.parser')
  images = bs4_apply.find_all('a')
  
  save_list = []

  for el in images:
    get_ref = el.get('href')
    save_list.append(get_ref)
    
  save_list = save_list[1:]

  correct_url_image = []
  # string in the list
  for s in save_list:
    if 'https' in s:
      # print(s)
      correct_url_image.append(s)

  print(correct_url_image[0])
  # SAVE THE IMAGE
  urllib.request.urlretrieve(correct_url_image[0], file_name)

In [175]:
scrape_images("https://commons.wikimedia.org/wiki/File:Crest-_Yosemite%3F_(17a34fd143a7446ea27c2138f52c2e16).jpg")

https://upload.wikimedia.org/wikipedia/commons/0/0f/Crest-_Yosemite%3F_%2817a34fd143a7446ea27c2138f52c2e16%29.jpg
